In [ ]:
pip install requests

In [ ]:
from datetime import datetime
import requests, json, pprint


In [ ]:
#modify these variables accordingly to your environment
'''
The ehr endpoints IPs  for each group are: 

 - 10.153.2.11	ORANGE
 - 10.153.2.12	RED
 - 10.153.2.13	BLUE
 - 10.153.2.14	GREEN
 - 10.153.2.15	YELLOW
 
 These patients have already been tested and should not be used for the test: 
 - IHSS_PAT_349 (ORANGE)
 - IHSS_PAT_106, IHSS_PAT_149 (RED)
 - IHSS_PAT_199 (BLUE)
 - IHSS_PAT_249 (GREEN)
 - IHSS_PAT_299 (YELLOW)

'''


#to be filled with the server address of the correspondent group
ehr_server_address = ''

#each student will execute this running example with a patient identifier taken from its cohort. 
assigned_patient_identifier = ''

#These variables should not be changed instead
ehr_server_port    =  8085
ihss_fhir_server_address = '10.153.2.16'
ihss_fhir_server_port    =  8090
ehr_server_auth_user = "ehrbase-user"
ehr_server_auth_password = "SuperSecretPassword"
ehr_subject_namespace = 'national'
template_id = 'Interhealth_cancer_registry'
ihss_fhir_server_base_url = f'http://{ihss_fhir_server_address}:{ihss_fhir_server_port}/fhir' 
ehr_server_base_uri = f'http://{ehr_server_address}:{ehr_server_port}/ehrbase/rest/openehr/v1/ehr'
aql_ehr_server_base_uri = f'http://{ehr_server_address}:{ehr_server_port}/ehrbase/rest/openehr/v1/query/aql'

In [ ]:
'''
PRELINIMARY STEP (1/2)
--------------------------------------------------------------------------------

Before starting the running example, let's check the laboratory exams that  the patient you have in charge has, 
saved in its own EHR. 
To do that, we need to retrieve the ehr id related to the patient, then perform an AQL query to retrieve the data. 
As you have seen yesterday, we have a different EHR for each patient. 
The EHRs have been created by assigning as subject_id of the EHR the IHSS patient identifier 
(second column of the dataset). 
The get_ehr_id function defined below performs a GET to the EHRBase server in order to retrive the EHR identifier, 
given the subkect identifier and the namespace.
'''


#get the ehr referred to the patient. N.B. All ehr have as subject id the 'hospital' identifier of the patient
def get_ehr_id(subject_id, subject_namespace):
    """
    Retrieve the EHR_ID of from the subject_id and the namespace. 
    This is needed to know in which EHR the new compositions should be added
    """
    ehr_get_uri = f"{ehr_server_base_uri}?subject_id={subject_id}&subject_namespace={subject_namespace}"
    print(ehr_get_uri)
    resp = requests.get(ehr_get_uri, auth=(ehr_server_auth_user, ehr_server_auth_password))
    if resp.status_code == 200: 
        return resp.json()["ehr_id"]["value"]
    return None

#call the method above and get the ehr id 
ehr_id = get_ehr_id(assigned_patient_identifier, ehr_subject_namespace)
print(ehr_id)


In [ ]:
'''
PRELINIMARY STEP (2/2)
--------------------------------------------------------------------------------

Now that we have the ehr_id, we can perform a get to check the laboratory data that the patient has. 
Below, there is a function to dynamically construct it, given the ehr_id
'''

def get_aql_query(ehr_id):
    #aql_query = f'select a/uid/value as uid from EHR e contains COMPOSITION a where e/ehr_id/value = \'{ehr_id}\''
    
    aql_query = """
    
    select
    e/ehr_id/value as EHR_id,
    c/uid/value as composition_id,
    c/context/other_context[at0001]/items[openEHR-EHR-CLUSTER.case_identification.v0, 'Registry identifier']/items[at0001]/value/value as registry_id,
    c/context/other_context[at0001]/items[openEHR-EHR-CLUSTER.case_identification.v0, 'IHSS identifier']/items[at0001]/value/value as IHSS_id,
    c/content[openEHR-EHR-SECTION.adhoc.v1, 'Laboratory tests']/items[openEHR-EHR-OBSERVATION.laboratory_test_result.v1]/data[at0001]/events[at0002]/data[at0003]/items[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]/items[at0024]/value/value as analyte_name,
    c/content[openEHR-EHR-SECTION.adhoc.v1, 'Laboratory tests']/items[openEHR-EHR-OBSERVATION.laboratory_test_result.v1]/data[at0001]/events[at0002]/data[at0003]/items[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]/items[at0024]/value/defining_code/code_string as code,
    c/content[openEHR-EHR-SECTION.adhoc.v1, 'Laboratory tests']/items[openEHR-EHR-OBSERVATION.laboratory_test_result.v1]/data[at0001]/events[at0002]/data[at0003]/items[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]/items[at0024]/value/defining_code/terminology_id/value as terminology_id,
    c/content[openEHR-EHR-SECTION.adhoc.v1, 'Laboratory tests']/items[openEHR-EHR-OBSERVATION.laboratory_test_result.v1]/data[at0001]/events[at0002]/data[at0003]/items[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]/items[at0001]/value/value as value,
    c/content[openEHR-EHR-SECTION.adhoc.v1, 'Laboratory tests']/items[openEHR-EHR-OBSERVATION.laboratory_test_result.v1]/data[at0001]/events[at0002]/data[at0003]/items[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]/items[at0004]/value/value as reference_range
from EHR e
contains COMPOSITION c
contains CLUSTER a_a[openEHR-EHR-CLUSTER.laboratory_test_analyte.v1]
    """ + f'where e/ehr_id/value = \'{ehr_id}\''
    
    return aql_query


aql_query = get_aql_query(ehr_id)

#execute the get and retrieve compositions. How many are them?

query_url = f'{aql_ehr_server_base_uri}?q={aql_query}'

results = requests.get(query_url, auth=(ehr_server_auth_user, ehr_server_auth_password))
pprint.pprint(results.json())


In [ ]:
# Now, interact with the FHIR server, with the purpose to get, for your assigned, patient, all the additional
# laboratory tests (observations) that the hospital has.
# The final puropose is to create a new composition for each retrieved observation and save it into your EHR
# To do that, the following steps shall be executed: 

#  -  Given a patient in your registry, get all the observations for that patient from the FHIR server. 
#  -  Scan the Bundle response and for each observation result, create the correspondent composition.  
#  -  Post the composition into the EHRBase server, then verify the proper insertion with an AQL Query 

In [ ]:
'''
Step 1 - Get observations for a patient
------------------------------------------------------------------------------
We call the utility function defined below, in order to query the FHIR server and get all the observations for the 
assigned patient.

'''

def get_observations_by_patient_identifier(patient_id):
    resource = "Observation"  # this is the resource that we have to query 
    # given the patient_id in input, the following line dinamically constructs the parameterized part.
    # if you remember the fist part of the tutorial, it is this get call: 
    # https://[fhir_server_address]/fhir/Observation?patient.identifier=[patient_identifier] 
    parameters = f'patient.identifier={patient_id}'
    request_uri = f'{ihss_fhir_server_base_url}/{resource}?{parameters}'
    response = requests.get(request_uri)
    return response.json()


observations_bundle = get_observations_by_patient_identifier(assigned_patient_identifier)
pprint.pprint(observations_bundle)

In [ ]:
'''
Step 2.1 - Define an utility funcion to create a composition,
----------------------------------------------------------------------------------
The function below takes in input some laboratory test information parameters, together with the related patient ID 
(subject identifier), and returns an instance of the corresponsent openEHR composition, whose nodes have been 
filled with the input laboratory test information parameters.

'''

def create_composition(subject_id, test_code, test_name, test_value, test_code_terminology, test_range_low, test_range_high, test_date):
    """
    Create an openEHR composition in FLAT format using the values passed in input
    """
    return {
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|code": test_code,
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_result:0/text_value": test_value,
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|terminology": test_code_terminology,
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/reference_range_guidance": f"{test_range_low}-{test_range_high}",
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/overall_test_status_timestamp": test_date,
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|value": test_name,
      "interhealth_cancer_registry/context/ihss_identifier/case_identifier": subject_id,
      "interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/test_name": "Blood cells count",

      "ctx/language": "en",
      "ctx/encoding|code": "UTF-8",
      "ctx/encoding|terminology": "IANA_character-sets",
      "ctx/composer_name": "IHSS",
      "ctx/width": "PT0S",
      "ctx/math_function|code": "145",
      "ctx/math_function|terminology": "openehr",
      "ctx/math_function|value": "minimum",
      "ctx/time": datetime.now().isoformat(),
      "ctx/category|code": "433",
      "ctx/category|terminology": "openehr",
      "ctx/category|value": "event",
      "ctx/territory|code": "DE",
      "ctx/territory|terminology": "ISO_3166-1"
    }


In [ ]:
'''
Step 2.2 - Scan the FHIR observation(s) results Bundle and fill the composition(s).
------------------------------------------------------------------------------

The FHIR response is a bundle of resources (observations).
We retrieve the response in Json format; it has some metadata on the top level. You can inspect it by forcing a print 
of the response itself. The 'entry' key is an array containing as more elements as the observations provided by the
FHIR server for the assigned patient are. Let's scan the entry array, and for each observation retrieve all the 
parameters required to create the correspondent openEHR composition (see create_composition function at step 2.1 . 
The required parameters values are located in different levels of each observation resource. For example: 

{'fullUrl': 'http://mobydick.crs4.it:10010/fhir/Observation/691',
 'resource': {'code': {'coding': [{'code': '41898006',
                                      |----------------------------------> test_code
                                   '
                                   display': 'Erythrocyte',
                                      |----------------------------------> test_name
                                   
                                   'system': 'http://snomed.info/sct'}]},
                                      |-----------------------------------> test_code_terminology
              'id': '691',
              'identifier': [{'value': 'IHSS_PAT_100-OBS_0'}],
              'interpretation': [{'coding': [{'code': 'N',
                                              'display': 'Normal',
                                              'system': 'http://terminology.hl7.org/CodeSystem/v3-ObservationInterpretation'}]}],
              'issued': '2016-05-16T06:09:42.000000',
                 |---------------------------------------------------------------- >test_date
              'meta': {'lastUpdated': '2022-10-05T15:14:28.960+00:00',
                       'source': '#SUedGkFibWV4cTW2',
                       'versionId': '1'},
              'referenceRange': [{'high': {'code': 'U/mm3',
                                           'system': 'http://unitsofmeasure.org',
                                           'unit': 'U/mm3',
                                           'value': 6000000},
                                              |------------------------------> test_range_high
                                  
                                  'low': {'code': 'U/mm3',
                                          'system': 'http://unitsofmeasure.org',
                                          'unit': 'U/mm3',
                                          'value': 4500000}}],
                                             |-------------------------------->test_range_low
              
              'resourceType': 'Observation',
              'status': 'final',
              'subject': {'reference': 'Patient/641'},
              'valueQuantity': {'code': 'U/mm3',
                                'system': 'http://unitsofmeasure.org',
                                'unit': 'U/mm3',
                                'value': 5919015}},
                                     |----------------------------> test_value
 
 'search': {'mode': 'match'}}


'''
#define a list where the created compositions will be saved
converted_compositions = list()

for observation in observations_bundle['entry']:
    #according to the position in the json structure of the Observation resource above, let's retrieve all information
    #required to create the OpenEHR composition: 
    test_code = observation['resource']["code"]["coding"][0]["code"]
    test_name = observation['resource']["code"]["coding"][0]["display"]
    test_code_terminology = observation['resource']["code"]["coding"][0]["system"]
    
    #Important: there are Tests whose result is a numeric value (i.e., Red Blood Cells) and there are some that 
    #Have a 'literal value' (i.e. Covid 19 result could be 'Positive' or Negative'. These two kinds of tests are
    #codified into the FHIR resource in a different structure: valueQuantity for numerics and valueCodeableConcept 
    #for literals. Notice that, obviously, for literals there isn't any range
    
    try:
        test_value = observation['resource']["valueQuantity"]["value"]
        test_range_low = observation['resource']["referenceRange"][0]["low"]["value"]
        test_range_high = observation['resource']["referenceRange"][0]["high"]["value"]
    
    except KeyError:
        test_value = observation['resource']["valueCodeableConcept"]["coding"][0]["display"]
        test_range_low = ''
        test_range_high = ''
    
    test_date = observation['resource']["issued"]
    
    print(test_code, test_name, test_code_terminology, test_value, test_date, test_range_low, test_range_high)
    
    #Now we can create the composotions, calling the conversion function defined at step 2.1. For convenenience, 
    #In order to easily execute the next steps, we save the composition in a list, defined at the top of this step.
    
    converted_composition = create_composition(assigned_patient_identifier, test_code, test_name, test_value, 
                                               test_code_terminology, test_range_low, test_range_high, test_date)
    
    converted_compositions.append(converted_composition)


pprint.pprint(converted_compositions)


In [ ]:
'''
Step 3.1 - Post the new compositions - Define a function to perform the POST request
------------------------------------------------------------------------------------
We have now completed all the steps to post the composition. The function below performs the POST query, taking
as input a composition in json format and the ehr identifier for the EHR where the composition itself will be 
posted, and the template identifier
'''

def post_composition(composition, ehr_id, template_id):     
    uri = f'http://{ehr_server_address}:{ehr_server_port}/ehrbase/rest/ecis/v1/composition/?format=FLAT&ehrId={ehr_id}&templateId={template_id}'
    response = requests.post(uri, json=composition, auth=(ehr_server_auth_user, ehr_server_auth_password))
    return response


In [ ]:
'''
Step 3.2 Post the new compositions: execute the POST for each of the compositions created in Step 2.2
-------------------------------------------------------------------------------------------------------
We have now completed all the steps to post the composition. The function below performs the POST query, taking
as input a composition in json format and the ehr identifier for the EHR where the composition itself will be 
posted, and the template identifier
'''

for composition in converted_compositions:
    response = post_composition(composition, ehr_id, template_id)
    print(response)
    pprint.pprint(response.json())

In [ ]:
'''
Step 4 - Issues at step 3.2? What happened?
---------------------------------------------------
Inspect the error log in the previous step. Why laboratory test codes has been refused? What can we do to 
fix the issue?
'''

In [ ]:
'''
Step 5 - Fix the composition
---------------------------------------------------
Which are the proper codes and terminology to use? How can we obtain the proper codes? 
The functions below allow you to fix the composition. The codes can be obtained by calling the terminology server, as 
you have seen in the examples in the first part of the running example.
'''

# First, complete these transcodings information. The values of the transcoding dictionary must be completed by 
# yourself, remembering how, given a code in a certain terminology, you can obtain the correspondence in another one. 
correct_test_code_terminology = 'LOINC'

transcodings = {
    '52501007': {
        'mapping_terminology_code' : '', #fill the correspondence 
        'mapping_terminology_name' : '' #fill the correspondence
    },
    '41898006': {
        'mapping_terminology_code' : '', #fill the correspondence 
        'mapping_terminology_name' : '' #fill the correspondence
    },
    '38082009': {
        'mapping_terminology_code' : '', #fill the correspondence 
        'mapping_terminology_name' : '' #fill the correspondence
    },
    '16378004': {
        'mapping_terminology_code' : '', #fill the correspondence 
        'mapping_terminology_name' : '' #fill the correspondence
    },
    '840533007': {
        'mapping_terminology_code' : '', #fill the correspondence 
        'mapping_terminology_name' : '' #fill the correspondence
    }
    
}

#Second, these transcodings are used by the function that fixes a composition

def fix_composition(composition): 
    wrong_terminology_test_code = composition["interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|code"]
    wrong_terminology_test_name = composition["interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|value"]
    correct_values = transcodings[wrong_terminology_test_code]
    #update the composition values with the proper codes
    composition["interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|code"] = correct_values['mapping_terminology_code']
    composition["interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|value"] = correct_values['mapping_terminology_name']
    composition["interhealth_cancer_registry/laboratory_tests/laboratory_test_result/any_event:0/laboratory_analyte_result/analyte_name|terminology"] = correct_test_code_terminology
    
    
#Third, fix all the compositions we had create before. Call the function above for each element of the compositions array 
#and save the new compositions in a new list 

correct_compositions = list()

for composition in converted_compositions:
    fix_composition(composition)
    correct_compositions.append(composition)


In [ ]:
'''
Step 6 - New POST attempt 
---------------------------------------------------
Retry to execute the post, and inspect results this time. Dit it success?
'''
for composition in correct_compositions:
    response = post_composition(composition, ehr_id, template_id)
    print(response)
    print(response.content)
    

In [ ]:
'''
Step 7 - SUCCESS!!!!!!!!!! 
---------------------------------------------------
Now we have solved the interoperability problems, and succesfully posted the compositions!
Now the ehr for the patient you had in charge had been enriched with the laboratory tests taken from the hospital
Congratulations!
'''

In [ ]:
'''
Step 8 - Perform an AQL query to inspect the added tests in the EHR  
------------------------------------------------------------------------

'''

results = requests.get(query_url, auth=(ehr_server_auth_user, ehr_server_auth_password))
pprint.pprint(results.json())
